In [1]:
import sys
sys.path

['C:\\ProgramData\\Ableton\\Live 10 Suite\\Resources\\MIDI Remote Scripts\\a_protocol_0\\jupyter',
 'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\thiba\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\thiba\\AppData\\Roaming\\Python\\Python39\\site-packages\\openapi_client-1.0.0-py3.9.egg',
 'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages',
 'C:\\ProgramData\\Ableton\\Live 10 Suite\\Resources\\MIDI Remote Scripts',
 'C:\\Users\\thiba\\Google Drive\\music\\dev\\Protocol0 System',
 'C:\\Users\\thiba\\Google Drive\\music\\dev\\toto3',
 'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\win32',
 'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39\